# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd 
import matplotlib
import numpy as np

In [2]:
df = pd.read_csv('zillow_data.csv')

In [3]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


# Step 2: Data Preprocessing

In [4]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [5]:
get_datetimes(df)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [6]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [16]:
df1 = df[df['City'] == 'Baltimore']
df2 = df[df['City'] =='Detroit']
df3 = df[df['City'] =='Houston']
df4 = df[df['City'] =='Miami']
df5 = df[df['City'] =='Philadelphia']
df6 = df[df['City'] =='Phoenix']
E = [df1,df2,df3,df4,df5,df6]

In [17]:
EmergingCitys = pd.concat(E,axis=0)

In [18]:
EmergingCitys

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
316,66825,21215,Baltimore,MD,Baltimore,Baltimore City,317,53500.0,53500.0,53400.0,...,59400,61400,63200,64100,65100,66800,67400,69500,72900,75000
441,66834,21224,Baltimore,MD,Baltimore,Baltimore City,442,49600.0,49400.0,49200.0,...,156900,161000,165800,168800,171100,173800,176400,177900,177500,176600
605,66828,21218,Baltimore,MD,Baltimore,Baltimore City,606,44700.0,44700.0,44700.0,...,73000,74500,76400,78100,81100,84800,88200,91800,96400,99900
677,66816,21206,Baltimore,MD,Baltimore,Baltimore City,678,73200.0,73300.0,73500.0,...,113100,115700,118400,119800,121100,123400,126600,129600,132400,135200
1004,66839,21229,Baltimore,MD,Baltimore,Baltimore City,1005,56000.0,56200.0,56500.0,...,77300,80000,83000,84400,85400,86100,86500,88400,92400,95600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8218,94767,85054,Phoenix,AZ,Phoenix,Maricopa,8219,185200.0,186300.0,187200.0,...,482800,486400,489800,494000,497400,500200,502900,503900,502700,502200
8631,94728,85012,Phoenix,AZ,Phoenix,Maricopa,8632,157100.0,158000.0,158900.0,...,530500,531200,530000,530600,533700,535300,533300,533100,537800,542400
9869,94761,85045,Phoenix,AZ,Phoenix,Maricopa,9870,190700.0,191900.0,193100.0,...,368400,370200,371200,372200,372900,373400,373800,374600,376500,378600
9871,66791,21131,Phoenix,MD,Baltimore,Baltimore,9872,278600.0,277500.0,276200.0,...,495100,497100,500400,502100,504000,505500,508200,510800,514000,515400


In [10]:
EmergingCitys.dropna(inplace=True)

In [19]:
EmergingCitys.drop(columns= 'RegionID',inplace=True)
EmergingCitys.drop(columns= 'SizeRank',inplace=True)

In [20]:
EmergingCitys

,RegionName,City,State,Metro,CountyName,1996-04,1996-05,1996-06,1996-07,1996-08,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
316,21215,Baltimore,MD,Baltimore,Baltimore City,53500.0,53500.0,53400.0,53300.0,53300.0,...,59400,61400,63200,64100,65100,66800,67400,69500,72900,75000
441,21224,Baltimore,MD,Baltimore,Baltimore City,49600.0,49400.0,49200.0,48900.0,48600.0,...,156900,161000,165800,168800,171100,173800,176400,177900,177500,176600
605,21218,Baltimore,MD,Baltimore,Baltimore City,44700.0,44700.0,44700.0,44700.0,44700.0,...,73000,74500,76400,78100,81100,84800,88200,91800,96400,99900
677,21206,Baltimore,MD,Baltimore,Baltimore City,73200.0,73300.0,73500.0,73600.0,73700.0,...,113100,115700,118400,119800,121100,123400,126600,129600,132400,135200
1004,21229,Baltimore,MD,Baltimore,Baltimore City,56000.0,56200.0,56500.0,56900.0,57300.0,...,77300,80000,83000,84400,85400,86100,86500,88400,92400,95600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8218,85054,Phoenix,AZ,Phoenix,Maricopa,185200.0,186300.0,187200.0,188200.0,189200.0,...,482800,486400,489800,494000,497400,500200,502900,503900,502700,502200
8631,85012,Phoenix,AZ,Phoenix,Maricopa,157100.0,158000.0,158900.0,159700.0,160500.0,...,530500,531200,530000,530600,533700,535300,533300,533100,537800,542400
9869,85045,Phoenix,AZ,Phoenix,Maricopa,190700.0,191900.0,193100.0,194300.0,195400.0,...,368400,370200,371200,372200,372900,373400,373800,374600,376500,378600
9871,21131,Phoenix,MD,Baltimore,Baltimore,278600.0,277500.0,276200.0,274800.0,273400.0,...,495100,497100,500400,502100,504000,505500,508200,510800,514000,515400


# Step 3: EDA and Visualization

In [13]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [21]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [22]:
melt_data(EmergingCitys)

,value
time,
1996-04-01,127723.369565
1996-05-01,127958.695652
1996-06-01,128198.369565
1996-07-01,128446.195652
1996-08-01,128698.913043
...,...
2017-12-01,268969.680851
2018-01-01,270249.468085
2018-02-01,271870.212766


# Step 5: ARIMA Modeling

# Step 6: Interpreting Results